In [1]:
import numpy
import sympy
from sympy.solvers import solve
from sympy import diff
from progressbar import progressbar
from math import e as euler
from math import log as ln

MAX_ITERATION = 1000
MAX_ITERATION_QUASE_NEWTON = 5000
MAX_ITERATIONITERATION_DICOTOMICA = 5000

In [2]:
#Var global
e = 0
c = 0
h = 0
nu = 0
lamb = 0
r_i = 0
r_j = 0
r_inc = 0

In [3]:
def function(x, f):
    if f == 1:
        return 1/2*(pow(x[0], 2) + (pow(x[1], 2)/3))
    if f == 2:
        return pow(euler, x[0]*x[1]*x[2]*x[3]*x[4])

In [4]:
def grad_function(x, f):
    if f == 1:
        p_1 = sum([nu[i] + 0.5*r_i[i]*(2*x[0]+2*x[1]-2) for i in range(len(c))])
        return [ p_1 + x[0], 
                 p_1 + (x[1]/3)]
    if f == 2:
        #_____________________X_0________________________________
        ret_x0 = pow(euler, x[0]*x[1]*x[2]*x[3]*x[4])*x[1]*x[2]*x[3]*x[4] 
        
        ret_x0_n0 = 2*nu[0]*x[0]
        ret_x0_n1 = 0
        ret_x0_n2 = 3*nu[2]*x[0]*x[0]
        
        ret_x0_r0 = 2*r_i[0]*x[0]*(x[0]*x[0]+x[1]*x[1]+x[2]*x[2]+x[3]*x[3]+x[4]*x[4]-10)
        ret_x0_r1 = 0
        ret_x0_r2 = 3*r_i[2]*x[0]*x[0]*(pow(x[0], 3)+pow(x[1], 3)+1)
        
        #_____________________X_1________________________________
        ret_x1 = pow(euler, x[0]*x[1]*x[2]*x[3]*x[4])*x[0]*x[2]*x[3]*x[4]
        
        ret_x1_n0 = 2*nu[0]*x[1]
        ret_x1_n1 = nu[1]*x[2]
        ret_x1_n2 = 3*nu[2]*x[1]*x[1]
        
        ret_x1_r0 = 2*r_i[0]*x[1]*(x[0]*x[0]+x[1]*x[1]+x[2]*x[2]+x[3]*x[3]+x[4]*x[4]-10)
        ret_x1_r1 = r_i[1]*x[2]*(x[1]*x[2]-5*x[3]*x[4])
        ret_x1_r2 = 3*r_i[2]*x[1]*x[1]*(pow(x[0], 3)+pow(x[1], 3)+1)
        
        #_____________________X_2________________________________
        ret_x2 = pow(euler, x[0]*x[1]*x[2]*x[3]*x[4])*x[0]*x[1]*x[3]*x[4]
        
        ret_x2_n0 = 2*nu[0]*x[2]
        ret_x2_n1 = nu[1]*x[1]
        ret_x2_n2 = 0
        
        ret_x2_r0 = 2*r_i[0]*x[2]*(x[0]*x[0]+x[1]*x[1]+x[2]*x[2]+x[3]*x[3]+x[4]*x[4]-10)
        ret_x2_r1 = r_i[1]*x[1]*(x[1]*x[2]-5*x[3]*x[4])
        ret_x2_r2 = 0
        
        #_____________________X_3________________________________
        ret_x3 = pow(euler, x[0]*x[1]*x[2]*x[3]*x[4])*x[0]*x[1]*x[2]*x[4]
        
        ret_x3_n0 = 2*nu[0]*x[3]
        ret_x3_n1 = -5*nu[1]*x[4]
        ret_x3_n2 = 0
        
        ret_x3_r0 = 2*r_i[0]*x[3]*(x[0]*x[0]+x[1]*x[1]+x[2]*x[2]+x[3]*x[3]+x[4]*x[4]-10)
        ret_x3_r1 = -5*r_i[1]*x[4]*(x[1]*x[2]-5*x[3]*x[4])
        ret_x3_r2 = 0
        
        #_____________________X_4________________________________
        ret_x4 = pow(euler, x[0]*x[1]*x[2]*x[3]*x[4])*x[0]*x[1]*x[2]*x[3]
        
        ret_x4_n0 = 2*nu[0]*x[4]
        ret_x4_n1 = -5*nu[1]*x[3]
        ret_x4_n2 = 0
        
        ret_x4_r0 = 2*r_i[0]*x[4]*(x[0]*x[0]+x[1]*x[1]+x[2]*x[2]+x[3]*x[3]+x[4]*x[4]-10)
        ret_x4_r1 = -5*r_i[1]*x[3]*(x[1]*x[2]-5*x[3]*x[4])
        ret_x4_r2 = 0
        return [ret_x0 + ret_x0_n0 + ret_x0_n1 + ret_x0_n2 + ret_x0_r0 + ret_x0_r1 + ret_x0_r2,
                ret_x1 + ret_x1_n0 + ret_x1_n1 + ret_x1_n2 + ret_x1_r0 + ret_x1_r1 + ret_x1_r2,
                ret_x2 + ret_x2_n0 + ret_x2_n1 + ret_x2_n2 + ret_x2_r0 + ret_x2_r1 + ret_x2_r2,
                ret_x3 + ret_x3_n0 + ret_x3_n1 + ret_x3_n2 + ret_x3_r0 + ret_x3_r1 + ret_x3_r2,
                ret_x4 + ret_x4_n0 + ret_x4_n1 + ret_x4_n2 + ret_x4_r0 + ret_x4_r1 + ret_x4_r2]

In [5]:
def restrictions(x, f):
    c_ = [] #Igualdade
    h_ = [] #Desigualdade
    if f == 1:
        c_.append(x[0]+x[1]-1)
    if f == 2:
        c_.append(x[0]*x[0]+x[1]*x[1]+x[2]*x[2]+x[3]*x[3]+x[4]*x[4]-10)
        c_.append(x[1]*x[3]-5*x[3]*x[4])
        c_.append(pow(x[0], 3)+pow(x[1], 3)+1)
    return [c_, h_]

In [6]:
def function_phi(x, f):
    #Passo 3
    p_nu_c = sum([nu[i]*c[i] for i in range(len(c))])
    p_lamb_h = sum([lamb[i]*h[i] for i in range(len(h))])
    p_r_c = sum([r_i[i]*pow(c[i], 2) for i in range(len(c))])/2
    p_r_h = sum([r_j[i]*pow(h[i], 2) for i in range(len(h))])/2
    return function(x, f) + p_nu_c + p_lamb_h + p_r_c + p_r_h

In [7]:
def busca_dicotomica(a_min, a_max, f, x_i, d_i):
    for k in range(MAX_ITERATIONITERATION_DICOTOMICA):
        if abs(a_max/a_min) < 1 + e*10 and k > 5:
                break
                
        lambda_k = ((a_min+a_max)/2)-e
        lambda_x = [x_i[j] + d_i[j] * lambda_k for j in range(len(x_i))]
        #Passo 3
        f_lambda = function_phi(lambda_x, f)
        
        mi_k = ((a_min+a_max)/2)+e
        mi_x = [x_i[j] + d_i[j] * mi_k for j in range(len(x_i))]
        #Passo 3
        f_mi = function_phi(mi_x, f)

        if f_lambda < f_mi:
            a_max = mi_k
        else:
            a_min = lambda_k
    return (a_min+a_max)/2

In [8]:
def DFP(input_D_i, input_s_i, input_y_i):
    s_i = numpy.array([[i] for i in input_s_i])
    y_i = numpy.array([[i] for i in input_y_i])
    D_i = numpy.array(input_D_i)
    p1 = numpy.true_divide(s_i.dot(s_i.transpose()), s_i.transpose().dot(y_i))
    div_p2 = y_i.transpose().dot(D_i).dot(y_i)
    nom_p2 = D_i.dot(y_i).dot(y_i.transpose()).dot(D_i)
    p2 = numpy.true_divide(nom_p2, div_p2)
    return D_i + p1 - p2

In [9]:
def metodo_quase_newton(x_i, f):
    D_i = numpy.identity(len(x_i))
    for i in range(MAX_ITERATION_QUASE_NEWTON):
        gradiente = grad_function(x_i, f)
        d_i = -D_i.dot(gradiente)
        
        if f == 1:
            alpha = busca_dicotomica(0.001, 2, f, x_i, d_i)

        if f == 2:
            alpha = busca_dicotomica(1, 2, f, x_i, d_i)

        antigo_x_i = x_i
        x_i = [x_i[k] + alpha*d_i[k] for k in range(len(x_i))]

        
        antigo_gradiente = gradiente
        gradiente = grad_function(x_i, f)

        convergiu = True
        for k in range(len(x_i)):
            if abs(antigo_x_i[k] - x_i[k]) > e:
                convergiu = False
        if convergiu:
            break
            
        s_i = [x_i[k] - antigo_x_i[k] for k in range(len(x_i))]
        y_i = [gradiente[k] - antigo_gradiente[k] for k in range(len(gradiente))]
        
        np_s_i = numpy.array(s_i)
        np_y_i = numpy.array(y_i)
        result = numpy.dot(np_s_i.transpose(), np_y_i)
        
        if result < 0:
            D_i = numpy.identity(len(x_i))
        
        if result >= 0:
            D_i = DFP(D_i, np_s_i, np_y_i)

    return x_i

In [10]:
def metodo_lagrangiano(x_k, f, r_i_, r_j_, r_inc_, e_):
    
    #Set global variables
    global e
    global c
    global h
    global nu
    global lamb
    global r_i
    global r_j
    global r_inc
    
    
    #Criando restrições
    temp = restrictions(x_k, f)
    c = temp[0]
    h = temp[1]
    
    e = e_
    r_inc = r_inc_
    
    #Passo 1
    nu = [0 for kk in range(len(c))]
    lamb = [0 for kk in range(len(h))]
    
    break_loop = False
    
    if type(r_i_) == type(list()):
        if len(r_i) != len(c):
            break_loop = True
    else:
        r_i = [r_i_ for kk in range(len(c))]
            
    if type(r_j_) == type(list()):
        if len(r_j) == len(c):
            break_loop = True
    else:
        r_j = [r_j_ for kk in range(len(h))]
    
    #Passo 2
    for k in progressbar(range(MAX_ITERATION)):
        if break_loop:
            print(f"Erro com tamanho de r_i ou r_j, esperado: {len(c)} e {len(h)}")
            break
        
        #Passo 5
        antigo_x_k = x_k
        
        #Passo 4
        x_k = metodo_quase_newton(x_k, f)

        #Passo 5
        convergiu = True
        for i in range(len(x_k)):
            if abs(antigo_x_k[i] - x_k[i]) > e/100 or k < 3:
                convergiu = False
        if convergiu:
            print(f"Numero de iterações foi o suficiente para convergir")
            break
            
        #Passo 6 
        for loop_i in range(len(r_i)):
            if c[loop_i] > 0.005 and c[loop_i] < -0.005:
                nu[loop_i] += r_i[loop_i]*c[loop_i]
                r_i[loop_i] += r_inc
            else:
                if f == 1:
                    r_i[loop_i] = k/((k%500) + 2)
                if f == 2:
                    r_i[loop_i] = 0.015*((k%100)+1)
        for loop_j in range(len(r_j)):
            if h[loop_k] > 0:
                lamb[loop_j] += r_j[loop_j]*h[loop_j]
                r_j[loop_j] += r_inc
        
        #Começo 3
        temp = restrictions(x_k, f)
        c = temp[0]
        h = temp[1]
#         if k%100 == 0:
#             print(x_k)
    print(x_k)

In [11]:
#x_k, f, r_i_, r_j_, r_inc_, e_
metodo_lagrangiano([0, 0], 1, 1, 1, 0.1, 0.001)

 54% (544 of 1000) |###########          | Elapsed Time: 0:00:39 ETA:   0:00:26

Numero de iterações foi o suficiente para convergir
[0.2452371102191033, 0.7341164965839392]


In [12]:
#x_k, f, r_i_, r_j_, r_inc_, e_
metodo_lagrangiano([-2, 2, 2, -1, -1], 2, 0.015, 0.015, 0.001, 0.001)

100% (1000 of 1000) |####################| Elapsed Time: 0:00:01 Time:  0:00:01


[-1.7164979160658371, 1.5953997542950331, 1.8318743836680684, -0.7677585916466587, -0.7677585916466586]
